In [1]:
import pandas as pd
import time

In [2]:
# pip install transformers torch accelerate

# 01 - Load the data

In [3]:
# Data Source
# Kaggle - https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [4]:
df = pd.read_csv("../Data/IMDBReviewsSubset.csv")
df.head()

,review,sentiment
0,"I am wanting to make a ""Holmes with Doors"" pun...",negative
1,This had high intellectual pretensions.The mai...,negative
2,"Tim Robbins is oddly benign here, cast as a ga...",negative
3,"Poorly acted, poorly written and poorly direct...",negative
4,While this isn't one of Miss Davies' very wors...,negative


In [5]:
df["sentiment"].value_counts()

sentiment
negative    1000
positive    1000
Name: count, dtype: int64

## 01-01 Sample reviews

In [6]:
# Examine a few reviews
df.loc[1, "review"]

'This had high intellectual pretensions.The main lead intends to give a "deep" "meaningful" rendering(with voice over for his frames of mind naturally) and he was certainly influenced by the fifties/sixties "method " -which,when the script and the direction were worthwhile did give stunning results (see Clift,Newman,Winters).But here the story is abysmal.Besides it moves too slow,you could edit at least 20 minutes -including pointless flashbacks-and the plot line would not be changed .At times ,it\'s very doubtful that Bruce Dern believes in his "Uncle "character and his portraying often verges on parody.An interesting side is only skimmed over:the relationship young boy/hero -if we admit that the hero is himself some kind of child- When he says to the young kid that he would let nobody do harm to him,some welcome tenderness emerges.But it\'s botched and only the final scene returns to it.<br /><br />Word to the wise:Take Foley\'s "at close range" instead:it has two great actors (Chris

In [7]:
df.loc[25, "review"]

"This movie was billed as a comedy and a mystery. It fails badly at both. The only mystery here is why would anybody make such a poorly constructed movie. The only comedy is the laugh I got when I saw how high the readers here ranked it. Could there be two movies with the same name? The movie I saw starred a girl with pretty blue eyes and a plot that wasn't there.<br /><br />"

# Sentiment Analysis with LLMs

In [8]:
# Tiny Llama

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [10]:
def analyze_sentiment(text):
    start = time.time()
    prompt = f"""
    Classify the sentiment of the following text as Positive or Negative. Give answer
    in one word, do not add anything else to response. 
    Text: "{text}"
    Sentiment:
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False
    )
    print ('input text is', text) 
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    end = time.time()
    print('elapsed time for sentiment analysis is', end - start)
    
    #return response.split("Sentiment:")[-1].strip()
    return response

In [11]:
# Example
ans = analyze_sentiment("I really enjoyed this game, it was fun!")
ans

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


input text is I really enjoyed this game, it was fun!
elapsed time for sentiment analysis is 98.47157979011536


'\n    Classify the sentiment of the following text as Positive or Negative. Give answer\n    in one word, do not add anything else to response. \n    Text: "I really enjoyed this game, it was fun!"\n    Sentiment:\n    1. Positive: "I really enjoyed this'

In [12]:
print (ans)


    Classify the sentiment of the following text as Positive or Negative. Give answer
    in one word, do not add anything else to response. 
    Text: "I really enjoyed this game, it was fun!"
    Sentiment:
    1. Positive: "I really enjoyed this


In [13]:
# Example
ans2 = analyze_sentiment("The service was slow and disappointing.")
ans2

input text is The service was slow and disappointing.
elapsed time for sentiment analysis is 99.13007545471191


'\n    Classify the sentiment of the following text as Positive or Negative. Give answer\n    in one word, do not add anything else to response. \n    Text: "The service was slow and disappointing."\n    Sentiment:\n    1. Positive: The service was slow and'

In [14]:
# Example
ans3 = analyze_sentiment("The service was very disappointing.")
ans3

input text is The service was very disappointing.
elapsed time for sentiment analysis is 98.08835697174072


'\n    Classify the sentiment of the following text as Positive or Negative. Give answer\n    in one word, do not add anything else to response. \n    Text: "The service was very disappointing."\n    Sentiment:\n    1. Positive: "The service was very'

In [15]:
# Example
ans4 = analyze_sentiment("Worst service at the restuarant. Never go.")
ans4

input text is Worst service at the restuarant. Never go.
elapsed time for sentiment analysis is 105.32765245437622


'\n    Classify the sentiment of the following text as Positive or Negative. Give answer\n    in one word, do not add anything else to response. \n    Text: "Worst service at the restuarant. Never go."\n    Sentiment:\n    1. Positive: "Worst service'

In [16]:
# GPT 2

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load GPT-2 Small (117M parameters)
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [18]:
# GPT-2 has no pad token by default
tokenizer.pad_token = tokenizer.eos_token

In [19]:
def analyze_sentiment(text):
    prompt = (
        "Classify the sentiment of the following text as "
        "Positive or Negative. \n\n"
        f"Text: {text}\n"
        "Sentiment:"
    )

    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the sentiment label
    #sentiment = response.split("Sentiment:")[-1].strip()
    return response

In [20]:
# Example
ans = analyze_sentiment("I really enjoyed this game, it was fun!")
ans

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Classify the sentiment of the following text as Positive or Negative. \n\nText: I really enjoyed this game, it was fun!\nSentiment: Positive\n\nComment:'

In [21]:
# Example
ans2 = analyze_sentiment("The service was slow and disappointing.")
ans2

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Classify the sentiment of the following text as Positive or Negative. \n\nText: The service was slow and disappointing.\nSentiment: The service was slow and'

In [22]:
# Example
ans3 = analyze_sentiment("The service was very disappointing.")
ans3

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Classify the sentiment of the following text as Positive or Negative. \n\nText: The service was very disappointing.\nSentiment: The service was very disappointing'

In [23]:
# Example
ans4 = analyze_sentiment("Worst service at the restuarant. Never go.")
ans4

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Classify the sentiment of the following text as Positive or Negative. \n\nText: Worst service at the restuarant. Never go.\nSentiment:\n\nThe sentiment of'

# Sentiment Analysis with Transformers

In [24]:
from transformers import pipeline

# Load a small, open-source sentiment analysis model
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

In [25]:
# Example texts
text_list = [
    "I love this product, it works perfectly!",
    "This is the worst experience I have ever had.",
    "The movie was okay, not great but not terrible."
]

# Run sentiment analysis
for texts in text_list:
    print(f"Text: {texts}")
    start = time.time()
    result = sentiment_analyzer(texts)
    end = time.time()
    print ('elapsed time for sentiment analysis is', end-start)     
    #print (result)
    print(f"Sentiment: {result[0]['label']}, Confidence: {result[0]['score']:.4f}\n")
    


Text: I love this product, it works perfectly!
elapsed time for sentiment analysis is 0.15555071830749512
Sentiment: POSITIVE, Confidence: 0.9999

Text: This is the worst experience I have ever had.
elapsed time for sentiment analysis is 0.041985273361206055
Sentiment: NEGATIVE, Confidence: 0.9998

Text: The movie was okay, not great but not terrible.
elapsed time for sentiment analysis is 0.052143096923828125
Sentiment: POSITIVE, Confidence: 0.9777

